In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
from dataloader import create_dataset

2025-05-03 20:14:33.679322: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-03 20:14:33.708323: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-03 20:14:33.708347: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-03 20:14:33.708374: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-03 20:14:33.714144: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-03 20:14:33.715318: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

{'Title': <tf.Tensor: shape=(), dtype=string, numpy=b'Guardians of the Galaxy'>, 'Genre': <tf.Tensor: shape=(), dtype=string, numpy=b'Action'>, 'Year': <tf.Tensor: shape=(), dtype=int64, numpy=2014>, 'Runtime': <tf.Tensor: shape=(), dtype=int64, numpy=121>, 'Rating': <tf.Tensor: shape=(), dtype=float64, numpy=8.1>, 'Metascore': <tf.Tensor: shape=(), dtype=float64, numpy=76.0>, 'Description': <tf.Tensor: shape=(), dtype=string, numpy=b'A group of intergalactic criminals are forced to work together to stop a fanatical warrior from taking control of the universe.'>}
{'Title': <tf.Tensor: shape=(), dtype=string, numpy=b'Guardians of the Galaxy'>, 'Genre': <tf.Tensor: shape=(), dtype=string, numpy=b'Adventure'>, 'Year': <tf.Tensor: shape=(), dtype=int64, numpy=2014>, 'Runtime': <tf.Tensor: shape=(), dtype=int64, numpy=121>, 'Rating': <tf.Tensor: shape=(), dtype=float64, numpy=8.1>, 'Metascore': <tf.Tensor: shape=(), dtype=float64, numpy=76.0>, 'Description': <tf.Tensor: shape=(), dtype=stri

2025-05-03 20:14:34.982912: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-03 20:14:35.005181: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
dataset = create_dataset()
movies = dataset.batch(8)

unique_titles = set()
unique_genres = set()

for batch in movies:
    # print(batch)
    titles = batch['Title'].numpy()  # tensor of strings
    genres = batch['Genre'].numpy()
    unique_titles.update(titles)
    unique_genres.update(genres)

unique_titles = np.array(list(unique_titles))
unique_genres = np.array(list(unique_genres))




In [3]:
movies = dataset.map(lambda x: x["Title"])
descriptions = dataset.map(lambda x: x["Description"])
years = np.concatenate(list(dataset.map(lambda x: x["Year"]).batch(100)))
ratings = np.concatenate(list(dataset.map(lambda x: x["Rating"]).batch(100)))
metascores = np.concatenate(list(dataset.map(lambda x: x["Metascore"]).batch(100)))

In [30]:
ratings[:10]

array([8.1, 8.1, 8.1, 7. , 7. , 7. , 7.3, 7.3, 7.2, 7.2])

In [31]:
metascores[:10]

array([76., 76., 76., 65., 65., 65., 62., 62., 59., 59.])

In [4]:
max_year = years.max()
min_year = years.min()

year_buckets = np.arange(
    min_year, max_year, 2
)


In [32]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10000

    self.genre_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_genres, mask_token=None),
        tf.keras.layers.Embedding(len(unique_genres) + 1, 32),
    ])
    self.year_embedding = tf.keras.Sequential([
        tf.keras.layers.Discretization(year_buckets.tolist()),
        tf.keras.layers.Embedding(len(year_buckets) + 1, 32),
    ])
    self.normalized_rating = tf.keras.layers.Normalization(
        axis=None
    )
    # self.normalized_metascore = tf.keras.layers.Normalization(
    #     axis=None
    # )
    self.desc_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.desc_embedding = tf.keras.Sequential([
      self.desc_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])


    self.desc_vectorizer.adapt(descriptions)
    self.normalized_rating.adapt(ratings)
    # self.normalized_metascore.adapt(metascores)

  def call(self, inputs):
    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([
        self.genre_embedding(inputs["Genre"]),
        self.year_embedding(inputs["Year"]),
        self.desc_embedding(inputs["Description"]),
        tf.reshape(self.normalized_rating(inputs["Rating"]), (-1, 1)),
        # tf.reshape(self.normalized_metascore(inputs["Metascore"]), (-1, 1)),
    ], axis=1)


In [41]:
class QueryModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(self, layer_sizes):
    """Model for encoding user queries.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = UserModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential([
      tf.keras.layers.Dense(layer_sizes[0])
    ])

    # self.dense_layers.build((None, 66))
    # dummy_input = tf.random.uniform((1, 66))
    # _ = self.dense_layers(dummy_input)

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [7]:
class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_titles,mask_token=None),
      tf.keras.layers.Embedding(len(unique_titles) + 1, 32)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.title_vectorizer.adapt(movies)

  def call(self, titles):
    return tf.concat([
        self.title_embedding(titles),
        self.title_text_embedding(titles),
    ], axis=1)

In [8]:
class CandidateModel(tf.keras.Model):
  """Model for encoding movies."""

  def __init__(self, layer_sizes):
    """Model for encoding movies.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    self.embedding_model = MovieModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential([
      tf.keras.layers.Dense(layer_sizes[0])
    ])

    # self.dense_layers.build((None, 64))
    # # Force weight creation
    # dummy_input = tf.random.uniform((1, 64))
    # _ = self.dense_layers(dummy_input)

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

    # embedding_dim = 32
    # self.dense_layers.build([None, embedding_dim])


  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [44]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, layer_sizes_query, layer_sizes_cand):
    super().__init__()
    self.query_model = QueryModel(layer_sizes_query)
    self.candidate_model = CandidateModel(layer_sizes_cand)
    # self.query_model = UserModel()
    # self.candidate_model = MovieModel()
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(32).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    query_embeddings = self.query_model({
        "Genre": features["Genre"],
        "Year": features["Year"],
        "Rating": features["Rating"],
        "Metascore": features["Metascore"],
        "Description": features["Description"]
    })
    print("Query_emb", query_embeddings)
    movie_embeddings = self.candidate_model(features["Title"])
    print("Movie_emb", movie_embeddings)

    return self.task(
        query_embeddings, movie_embeddings, compute_metrics=not training)

In [45]:
shuffled = dataset.shuffle(1000, seed=42, reshuffle_each_iteration=False)
test = shuffled.take(100)
cached_train = dataset.batch(32).cache()
cached_test = test.batch(32).cache()


In [33]:
cand_model = CandidateModel([32])
quer_model = QueryModel([32])

/home/ds_students/anaconda3/envs/moodfli/lib/python3.10/site-packages/numpy/core/numeric.py:2468: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [34]:
for row in cached_test.take(1):
    print(quer_model({
        "Genre": row["Genre"],
        "Year": row["Year"],
        "Rating": row["Rating"],
        "Metascore": row["Metascore"],
        "Description": row["Description"]
    }))

Feature:  tf.Tensor(
[[ 1.0444511e-02 -3.5711028e-02  1.5444588e-02 ... -3.9134803e-03
   6.8814028e-03 -2.2315037e+00]
 [ 1.2819912e-02 -1.5509836e-03  3.2763369e-03 ...  1.5111574e-03
   4.6860855e-03 -9.2937207e-01]
 [-9.1033578e-03 -1.6571712e-02  2.1583501e-02 ... -8.7014064e-03
   5.3433847e-04 -4.9532822e-01]
 ...
 [ 8.6917505e-03  4.2821094e-04 -2.1085966e-02 ... -8.1995176e-03
   2.4131734e-03  1.3493577e+00]
 [ 1.9613612e-02 -3.2651234e-02  1.0578789e-02 ...  1.2999254e-03
   6.9552218e-03  5.8978128e-01]
 [ 8.6917505e-03  4.2821094e-04 -2.1085966e-02 ...  7.0575433e-04
   3.1685568e-03  1.3493577e+00]], shape=(32, 97), dtype=float32)
tf.Tensor(
[[-0.07572958 -0.34633428 -0.60807157 ...  0.29320794  0.12286784
  -0.48907983]
 [-0.0369146  -0.1526255  -0.22835782 ...  0.10904159  0.02482696
  -0.1893226 ]
 [ 0.01591535 -0.06743775 -0.15524805 ...  0.06287812  0.02239685
  -0.11111928]
 ...
 [ 0.071205    0.26479918  0.3562327  ... -0.14463918 -0.0842466
   0.30393445]
 [ 0.029

In [46]:
num_epochs = 200

model = MovielensModel([64, 32], [32])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.01))

one_layer_history = model.fit(
    cached_train,
    validation_data=cached_test,
    validation_freq=5,
    epochs=num_epochs,
    verbose=1)

/home/ds_students/anaconda3/envs/moodfli/lib/python3.10/site-packages/numpy/core/numeric.py:2468: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


Epoch 1/200
Query_emb Tensor("query_model_10/sequential_75/dense_42/BiasAdd:0", shape=(None, 32), dtype=float32)
Movie_emb Tensor("candidate_model_11/sequential_78/dense_44/BiasAdd:0", shape=(None, 32), dtype=float32)
Query_emb Tensor("query_model_10/sequential_75/dense_42/BiasAdd:0", shape=(None, 32), dtype=float32)
Movie_emb Tensor("candidate_model_11/sequential_78/dense_44/BiasAdd:0", shape=(None, 32), dtype=float32)
80/80 [==============================] - 1s 2ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 110.3161 - regularization_loss: 0.0000e+00 - total_loss: 110.3161
Epoch 2/200
80/80 [==============================] - 0s 2ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categor

In [47]:
accuracy = one_layer_history.history["val_factorized_top_k/top_100_categorical_accuracy"][-1]
print(f"Top-100 accuracy: {accuracy:.2f}.")

Top-100 accuracy: 0.41.


In [49]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.candidate_model)))
)

# Get recommendations.
for row in test.batch(1).take(10):
    print(row)
    _, titles = index(row)
    print(f"Recommendations: {titles[0, :10]}")

{'Title': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Dead Awake'], dtype=object)>, 'Genre': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Horror'], dtype=object)>, 'Year': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([2016])>, 'Runtime': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([99])>, 'Rating': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([4.7])>, 'Metascore': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([nan])>, 'Description': <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'A young woman must save herself and her friends from an ancient evil that stalks its victims through the real-life phenomenon of sleep paralysis.'],
      dtype=object)>}
Recommendations: [b'Office Christmas Party' b'Pet' b'Pet' b'Bride Wars' b'Bride Wars'
 b"Mother's Day" b"Mother's Day" b'Folk Hero & Funny Guy' b'Collide'
 b'Collide']
{'Title': <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'Percy Jackson & the Olympians: The Lightning Thief'],
      dtype=obj